In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import torch
import torchvision
import torch.nn as nn
import torchvision.models as models
import numpy as np
import pandas as pd
import clearml
import matplotlib.pyplot as plt
import seaborn

import warnings
warnings.filterwarnings(action='ignore')
%matplotlib inline

torch.cuda.is_available()

True

In [2]:
import ssl
import certifi

ssl._create_default_https_context = ssl.create_default_context
ssl._create_default_https_context.cafile = certifi.where()

In [3]:
task = clearml.Task.init(project_name="Caption generator", task_name="Selecting encoder", tags=['encoder', 'feature_extraction'])
logger = task.get_logger()

ClearML Task: overwriting (reusing) task id=22755effd0fd491bbb9de938e9065e17
ClearML results page: https://app.clear.ml/projects/22856c5ce3ac4602a83e933c56a4600a/experiments/22755effd0fd491bbb9de938e9065e17/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


## Encoders factory

In [4]:
class BaseEncoder(nn.Module):
    def __init__(self, embedding_dim=None):
        super().__init__()
        self.embedding_dim=None
        self.features_dim=None

    def forward(self, x):
        pass

    def get_features_dim(self):
        return self.features_dim

In [5]:
class ResNet50(BaseEncoder):
    def __init__(self, embedding_dim=256, pretrained=True):
        super().__init__(embedding_dim)
        model = models.resnet50(pretrained=pretrained)
        self.features = nn.Sequential(*list(model.children())[:-1])

        self.feature_dim = 2048

    def forward(self, x):
        features = self.features(x)
        out = torch.flatten(features, 1)

        return out

In [6]:
class ResNet101(BaseEncoder):
    def __init__(self, embedding_dim=256, pretrained=True):
        super().__init__(embedding_dim)
        model = models.resnet101(pretrained=pretrained)
        self.features = nn.Sequential(*list(model.children())[:-1])

        self.feature_dim = 2048

    def forward(self, x):
        features = self.features(x)
        out = torch.flatten(features, x)

        return out

In [7]:
class EfficientNet(BaseEncoder):
    def __init__(self, embedding_dim=256, model_name='efficientnet_b0', pretrained=True):
        super().__init__(embedding_dim)
        weights = 'DEFAULT' if pretrained else None
        if model_name == 'efficientnet_b0':
            model = models.efficientnet_b0(weights=weights)
        elif model_name == 'efficientnet_b4':
            model = models.efficientnet_b4(weights=weights)
        else:
            raise ValueError('Unknown model (efficient net)')

        self.feature_dim = 1280 if 'b0' in model_name else 1792
        self.features = nn.Sequential(*list(model.children())[:-1])

    def forward(self, x):
        features = self.features(x)
        out = torch.flatten(features, 1)

        return out

In [8]:
class EncoderFactory:
    @staticmethod
    def create_encoder(encoder_name, embedding_dim=256, pretrained=True):
        all_encoders = {
            "resnet50": ResNet50,
            "resnet101": ResNet101,
            "efficientnet_b0": lambda **kwargs: EfficientNet(model_name="efficientnet_b0", **kwargs),
            "efficientnet_b4": lambda **kwargs: EfficientNet(model_name="efficientnet_b4", **kwargs)
        }
        selected_encoder = all_encoders[encoder_name](
            embedding_dim=embedding_dim,
            pretrained=pretrained
        )

        return selected_encoder

In [9]:
import os
import ssl
import urllib.request

ssl._create_default_https_context = ssl._create_unverified_context

import urllib.request as ur
ur._opener = None
os.environ['CURL_CA_BUNDLE'] = ''

In [10]:
resnet50 = EncoderFactory.create_encoder("resnet50")

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/haritonn/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100.0%
